#Data Wrangling with CSV, Excel and JSON data
In this lesson, we discuss how to retrieve data from various formats.  Data lives in many formats: CSV,Excel and JSON just to name a few.  Below we show how to use pyth
on to extract data from these various formats.

##Data Wrangling CSV files
CSV or "comma-seperated values" is the most lightweight form of storing data. Each value is seperated by a comma as a delimiter and optionally there may be a header line to describe the values as a column format.  Another lightweight format is the "tab-seperated values" in the which values are delimited by the tab character.  The *csv* package can be used for both of these formats.  Our first code example shows us how to read in data using the *csv.reader* object.  This object comes with a iterator which is called by *next()* and used to read an entire line from the csv file.  The code below appends each line of the csv file into a list data type.   We show what the first line after the header as well as one of the entries from the header(*name*).

In [1]:
import csv
import os

DATADIR = ""
DATAFILE = "745090.csv"


def parse_file(datafile):
    name = ""
    data = []
    with open(datafile,'rb') as f:
        reader = csv.reader(f)
        l = reader.next()
        name = l[1]
        reader.next()
        for row in reader:
            data.append(row)
    # Do not change the line below
    #print data[0][1]
    return (name, data)

In [2]:
name,data = parse_file(DATAFILE)
print data[0]
print name

['01/01/2005', '01:00', '0', '0', '0', '2', '0', '0', '2', '0', '0', '2', '0', '0', '2', '0', '0', '2', '0', '0', '2', '0', '0', '2', '0', '3', 'E', '9', '3', 'E', '9', '8.0', 'A', '7', '6.0', 'A', '7', '87', 'A', '7', '1013', 'A', '7', '150', 'A', '7', '2.1', 'A', '7', '16100', 'A', '7', '77777', 'A', '7', '1.1', 'E', '8', '0.099', 'F', '8', '0.160', 'F', '8', '0', '1', 'A', '7']
MOUNTAIN VIEW MOFFETT FLD NAS


A more interesting format is generated by the **DictReader** object.  This creates key-value pairs for entry in the the data set.  This uses the header entries as the key and assigns the value field to the entry that matches that row-column location.  This format can be useful for organizing the data in a more complex format such as XML or JSON.  Below is the code to read in a csv file into a DictReader object.  Here we show how an entry in this format would look.  Notice that each line has created a dictonary object.  thus the DictReady is a list of Dictonaries.

In [3]:
import pprint
datafile = "beatles-diskography.csv"
with open(datafile,'rb') as f:
    d_ = csv.DictReader(f)
    line = d_.next()
    fields = d_.fieldnames
    pprint.pprint(line)
    print fields

{'BPI Certification': 'Gold',
 'Label': 'Parlophone(UK)',
 'RIAA Certification': 'Platinum',
 'Released': '22 March 1963',
 'Title': 'Please Please Me',
 'UK Chart Position': '1',
 'US Chart Position': '-'}
['Title', 'Released', 'Label', 'UK Chart Position', 'US Chart Position', 'BPI Certification', 'RIAA Certification']


##Extracting Data from Excel Files
Extracting data from excel is done through the **xlrd** python package.  This package allow you to read Excel files in both the *xls* format as well as the *xlsx* format.  Below we show the various function that can be used to extract entries from the Excel workbook.  Below are examples of how to read a specific sheet, retrieve a cell's type/data, and how to convert dates into a tuple format.


In [5]:
import xlrd
#from zipfile import ZipFile
datafile = "2013_ERCOT_Hourly_Load_Data.xls"
def open_zip(datafile):
    with ZipFile('{0}.zip'.format(datafile), 'r') as myzip:
        myzip.extractall()
def parse_file(datafile):
    workbook = xlrd.open_workbook(datafile)
    sheet = workbook.sheet_by_index(0)

    ### example on how you can get the data
    #sheet_data = [[sheet.cell_value(r, col) for col in range(sheet.ncols)] for r in range(sheet.nrows)]

    ### other useful methods:
    # print "\nROWS, COLUMNS, and CELLS:"
    # print "Number of rows in the sheet:", 
    # print sheet.nrows
    # print "Type of data in cell (row 3, col 2):", 
    # print sheet.cell_type(3, 2)
    # print "Value in cell (row 3, col 2):", 
    # print sheet.cell_value(3, 2)
    # print "Get a slice of values in column 3, from rows 1-3:"
    # print sheet.col_values(3, start_rowx=1, end_rowx=4)

    # print "\nDATES:"
    # print "Type of data in cell (row 1, col 0):", 
    # print sheet.cell_type(1, 0)
    # exceltime = sheet.cell_value(1, 0)
    # print "Time in Excel format:",
    # print exceltime
    # print "Convert time to a Python datetime tuple, from the Excel float:",
    # print xlrd.xldate_as_tuple(exceltime, 0)
    
    
    data = {
            'maxtime': (0, 0, 0, 0, 0, 0),
            'maxvalue': 0,
            'mintime': (0, 0, 0, 0, 0, 0),
            'minvalue': 0,
            'avgcoast': 0
    }
    l = sheet.col_values(1, start_rowx=1, end_rowx=sheet.nrows)
    max_value = max(l)
    row = l.index(max_value) + 1
#    print row, max_value
    maxtime = sheet.cell_value(row,0)
 #   print maxtime
    data['maxtime'] = xlrd.xldate_as_tuple(maxtime,0)
    data['maxvalue'] = sheet.cell_value(row,1)
    min_value = min(l)
    row = l.index(min_value)
    mintime = sheet.cell_value(row,0)
    data['mintime'] = xlrd.xldate_as_tuple(mintime,0)
    data['minvalue'] = sheet.cell_value(row,1)
    data['avgcoast'] = sum(l) / float(len(l))  
    return data

pprint.pprint(parse_file(datafile))

{'avgcoast': 10976.933460679751,
 'maxtime': (2013, 8, 13, 17, 0, 0),
 'maxvalue': 18779.025510000003,
 'mintime': (2013, 2, 3, 3, 0, 0),
 'minvalue': 6677.16427800001}


##Extracting Data from a JSON format
*JSON* data is one of the more flexible formats to work with. It gives you the ability to organize the data in key-value pairs for in non-unique formats for each JSON object.  This ability becomes handy when there is one field that may pertain to one JSON object, but not all of the others.  JSON objects can also be nested inside of other JSON objects and some fields could have an single value, or and array value paired with a specific key.  The code below shows how we you use the API for the *musicbrainz* site to retrieve data based on specific HTTP requests.  We use the **requests** package to make the HTTP requests and we use the **json** package to print out the results.  JSON data pairs very well with Python dictonaries.

In [6]:
# To experiment with this code freely you will have to run this code locally.
# We have provided an example json output here for you to look at,
# but you will not be able to run any queries through our UI.
import json
import requests


BASE_URL = "http://musicbrainz.org/ws/2/"
ARTIST_URL = BASE_URL + "artist/"

query_type = {  "simple": {},
                "atr": {"inc": "aliases+tags+ratings"},
                "aliases": {"inc": "aliases"},
                "releases": {"inc": "releases"}}


def query_site(url, params, uid="", fmt="json"):
    params["fmt"] = fmt
    r = requests.get(url + uid, params=params)
    print "requesting", r.url

    if r.status_code == requests.codes.ok:
        return r.json()
    else:
        r.raise_for_status()


def query_by_name(url, params, name):
    params["query"] = "artist:" + name
    return query_site(url, params)


def pretty_print(data, indent=4):
    if type(data) == dict:
        print json.dumps(data, indent=indent, sort_keys=True)
    else:
        print data


def main():
    results = query_by_name(ARTIST_URL, query_type["simple"], "Queen")
    #pretty_print(results)
    print len(results["artists"])

    artist_id = results["artists"][10]["id"]
    print "\nARTIST:"
    pretty_print(results["artists"][10])

    artist_data = query_site(ARTIST_URL, query_type["releases"], artist_id)
    releases = artist_data["releases"]
    print "\nONE RELEASE:"
    #pretty_print(releases[0], indent=2)
    release_titles = [r["title"] for r in releases]

    print "\nALL TITLES:"
    for t in release_titles:
        print t
main()

requesting http://musicbrainz.org/ws/2/artist/?query=artist%3AQueen&fmt=json
25

ARTIST:
{
    "id": "b9de990e-2ece-42dd-827c-30c4ca4a99a4", 
    "life-span": {
        "ended": null
    }, 
    "name": "Queen V", 
    "score": "62", 
    "sort-name": "Queen V"
}
requesting http://musicbrainz.org/ws/2/artist/b9de990e-2ece-42dd-827c-30c4ca4a99a4?fmt=json&inc=releases

ONE RELEASE:

ALL TITLES:


This code below shows how we first make a initial request for an artist.  Once we have that artists's unique id, we make a post request for that artist's aliases.  We then pretty_print the data using the *json* dump function which is defined in the code above.

In [7]:
results = query_by_name(ARTIST_URL, query_type["simple"], "First Aid Kit")
pretty_print(results)
count = 0
for value in results["artists"]:
    if value["name"] == "First Aid Kit":
        count = count + 1
print count
#pretty_print(results["artists"][1])
artist_id = results["artists"][1]["id"]
artist_data = query_site(ARTIST_URL, query_type["aliases"], artist_id)

pretty_print(artist_data)

requesting http://musicbrainz.org/ws/2/artist/?query=artist%3AFirst+Aid+Kit&fmt=json
{
    "artists": [
        {
            "area": {
                "id": "23d10872-f5ae-3f0c-bf55-332788a16ecb", 
                "name": "Sweden", 
                "sort-name": "Sweden"
            }, 
            "country": "SE", 
            "disambiguation": "Swedish folk band", 
            "id": "373faa02-74d7-4b1d-9b47-7574ad510f8d", 
            "life-span": {
                "begin": "2007", 
                "ended": null
            }, 
            "name": "First Aid Kit", 
            "score": "100", 
            "sort-name": "First Aid Kit", 
            "tags": [
                {
                    "count": 1, 
                    "name": "remember"
                }
            ], 
            "type": "Group"
        }, 
        {
            "area": {
                "id": "471c46a7-afc5-31c4-923c-d0444f5053a4", 
                "name": "Spain", 
                "sort-name": "Spain"
  